
# Discover Cost-Efficient AI Customer Service Agents with NVIDIA Data Flywheel Blueprint
[![ Click here to deploy.](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-2wggjBvDlVp4pLQD8ytZySh5m8W)

In this notebook, you will learn learn how to use the Data Flywheel Foundational Blueprint to continuously discover and promote more cost-efficient agents for an [AI virtual customer service assistant](https://build.nvidia.com/nvidia/ai-virtual-assistant-for-customer-service).

### Data Flywheel Blueprint

![dfw_arch](../docs/images/data-flywheel-blueprint.png)


### AI Virtual Assistant for Customer Service

The primary customer service agent in the AI Virtual Assistant uses tool calling to route user queries to specialized assistants, including: 

- Product Q&A
- Order status verification
- Returns processing
- Small talk and casual engagement

These interactions generate logs and tool-calling data that you can use as both evaluation benchmarks and training data. In this tutorial, you'll use this information to drive the flywheel process, fine-tuning smaller LLMs (such as `meta/llama-3.2-1B-instruct`, `meta/llama-3.2-3B-instruct`, `meta/llama-3.1-8B-instruct`) to match accuracy of the currently deployed model (`meta/llama-3.3-70B-instruct`).



## Interfacing with the Blueprint

The following diagram illustrates how admin tools and applications interact with the Flywheel Blueprint, which orchestrates logging, processing, and model management to enable continuous optimization.

![Arch](./arch.png)

### Contents 

0. [Data Flywheel Setup](#0)
1. [Load Sample Data](#1)
2. [Create a Flywheel Job](#2)
3. [Monitor Job Status](#3)
4. [Optional: Show Continuous Improvement](#4)

<a id="0"></a>
## Data Flywheel Setup

### General Setup

In general, you can start the Data Flywheel service by following the instructions provided in the notebook [README.md](https://github.com/NVIDIA-AI-Blueprints/data-flywheel/blob/main/notebooks/README.md).

### Brev Launchable Setup

If you’re running this notebook from a [Brev Launchable](https://brev.nvidia.com/launchable/deploy/now?launchableID=env-2wggjBvDlVp4pLQD8ytZySh5m8W), please complete the following setup steps before proceeding:

**Step 1**: Set NGC API key following the instructions at [Generating NGC API Keys](https://docs.nvidia.com/ngc/gpu-cloud/ngc-private-registry-user-guide/index.html#generating-api-key).

Set your docker registry credentials to allow MLRun CE installation. For more information see [MLRun CE Installation](https://docs.mlrun.org/en/latest/install/kubernetes.html#installing-the-chart).

In [ ]:
import os
os.environ['NGC_API_KEY'] = '<your_ngc_api_key>'
os.environ['DOCKER_SERVER'] = '<your_docker_server>'
os.environ['DOCKER_USERNAME'] = '<your_docker_username>'
os.environ['DOCKER_PASSWORD'] = '<your_docker_password>'
os.environ['DOCKER_REGISTRY_URL'] = '<your_docker_registry_url>'

**Step 2**: Clone the data flywheel repo and fetch data files.

In [ ]:
%%bash
git clone https://github.com/NVIDIA-AI-Blueprints/data-flywheel.git
cd data-flywheel
sudo apt-get update && sudo apt-get install -y git-lfs
git lfs install
git-lfs pull

**Step 3**: Set up paths and installs python dependencies for notebook.

In [ ]:
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir / "data-flywheel"
data_dir = project_root / "data"
sys.path.insert(0, str(project_root))
os.chdir(project_root)
print(f"Working directory changed to: {Path.cwd()}")

user_site = Path.home() / ".local" / "lib" / f"python{sys.version_info.major}.{sys.version_info.minor}" / "site-packages"
if str(user_site) not in sys.path:
    sys.path.append(str(user_site))
    print(f"Added user site-packages to sys.path: {user_site}")

%pip install --user elasticsearch==8.17.2 pydantic-settings>=2.9.1 pandas>=2.2.3 matplotlib==3.10.3

**Step 4**: Update `config/config.yaml` to use remote LLM as judge. By default, data flywheel blueprint deploys `LLama-3.3-70B-instruct` locally for LLM as a judge, which requires 4 GPUs. But for the launchable, we will choose the remote LLM judge and use the `LLama-3.3-70B-instruct` NIM hosted on [build.nvidia.com](https://build.nvidia.com/meta/llama-3_3-70b-instruct).

By default, only `Llama-3.2-1b-instruct` will be used in the flywheel but you can uncomment other models in the yaml file to include in the flywheel run. You can also change other config settings such as data split and training hyperparameters as desired



In [ ]:
import re

config_path = project_root / "config" / "config.yaml"
with open(config_path, "r") as f:
    original_yaml = f.read()

llm_judge_config_block = """llm_judge_config:
  type: "remote"
  url: "https://integrate.api.nvidia.com/v1/chat/completions"
  model_id: "meta/llama-3.3-70b-instruct"
  api_key_env: "NGC_API_KEY"
"""
updated_yaml = re.sub(
    r"llm_judge_config:.*?(?=\n\w|\Z)",  # stops at next top-level key
    llm_judge_config_block,
    original_yaml,
    flags=re.DOTALL
)

with open(config_path, "w") as f:
    f.write(updated_yaml)

**Step 5**: Start data flywheel service, which involves first deploying the Nemo Microservices Platform and then bring up the data flywheel service via docker compose. This step take some time.

In [ ]:
%%bash
set -e

log() {
  echo -e "\033[1;32m[INFO]\033[0m $1"
}

echo "$NGC_API_KEY" | docker login nvcr.io -u '$oauthtoken' --password-stdin
chmod +x scripts/deploy-nmp.sh scripts/run.sh scripts/mlrun.sh

log "Starting Nemo Microservices Platform (NMP) deployment..."
./scripts/deploy-nmp.sh >> flywheel_deploy.log 2>&1
log "NMP deployed successfully!"

log "Starting data flywheel service..."
./scripts/run.sh >> flywheel_deploy.log 2>&1
log "Data flywheel service started successfully!"

log "Starting mlrun services..."
./scripts/mlrun.sh >> flywheel_deploy.log 2>&1
log "MLRun services deployed successfully!"

---

<a id="1"></a>
## Step 1: Load Sample Data


First, we need to import required libraries and configure pandas display options for better readability in notebook outputs.

In [ ]:
import sys
from pathlib import Path
import requests
import time
from datetime import datetime
import json
import pandas as pd
from IPython.display import display, clear_output

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Width of the display in characters
pd.set_option('display.max_colwidth', None)  # Show full content of each cell

Use the provided sample dataset from AI Virtual Assistant (`aiva`) (`data/aiva_primary_assistant_dataset.jsonl`) to simulate real user logs captured while an agentic customer service agent application is running. Each data point has the following schema:

| Field        | Type               | Description                                                         |
|--------------|--------------------|---------------------------------------------------------------------|
| `timestamp`  | `int` (epoch secs) | Time the request was issued                                         |
| `workload_id`| `str`              | Stable identifier for the logical task / route / agent node         |
| `client_id`  | `str`              | Identifier of the application or deployment that generated traffic  |
| `request`    | `dict`             | Exact [`openai.ChatCompletion.create`](https://platform.openai.com/docs/api-reference/chat/create) payload received by the model |
| `response`   | `dict`             | Exact `ChatCompletion` response returned by the model               |

The `reuqest` uses the OpenAI `ChatCompletions` request format and contains the following attributes:

- `model` includes the Model ID used to generate the response.
- `messages` includes a `system` message as well as a `user` query.
- `tools` includes a list of functions and parameters available to the LLM to choose from, as well as their parameters and descriptions.

In [ ]:
DATA_PATH = data_dir / "aiva_primary_assistant_dataset.jsonl"

!head -n1 {DATA_PATH} | jq

The data points generated by AI Virtual Assistant in response to user queries are considered **ground truth**. 

Ground truth data points are used to **evaluate** and **customize** more efficient models that can perform similarly to the current model. This customization process is analogous to a student-teacher distillation setup, where synthetic data generated from the teacher model is used to fine-tune a student model.

Next, we'll load the data into Elasticsearch using a helper method `load_data_to_elasticsearch`, making it accessible to the Flywheel Orchestrator.

In [ ]:
from src.scripts.load_test_data import load_data_to_elasticsearch

load_data_to_elasticsearch(file_path=DATA_PATH)

---

<a id="2"></a>
## Step 2: Create a Data Flywheel Workflow from MLRun

Now it's time to move to your Orchestrated environment—MLRun.

To get into the MLRun environment, please follow the instructions below:

1. Go to your Brev Launchable and click on the **Access** tab.
2. In the **Access** tab, click on the **Share A Service** button to create the following services:
   1. **mlrun-jupyter**: Port `30040`. This is the JupyterLab environment where you can execute mlrun workflows.
   2. **mlrun-ui**: Port `30060`. This is the MLRun UI where you can monitor and manage your project with all the runs, functions, and artifacts.
   3. **nuclio-dashboard**: Port `30050`. This is the Nuclio dashboard where you can monitor and manage your realtime functions.
3. After creating the services, open each service in a new tab and continue from **mlrun-jupyter**.
4. In the **mlrun-jupyter** tab, open a new terminal and clone the `mlrun-data-flywheel`. You can use the following command:
   ```bash
   git clone https://github.com/yonishelach/data-flywheel.git --branch development
   ```

![Services](./services.png)